# Bài tập 2

Đặng Hồng Minh - 1712096

## Câu 1 (1 điểm)

In [1]:
import numpy as np
import math

In [2]:
nus = np.random.binomial(10, 0.5, (100000, 1000))/10
v_cmin = np.min(nus,axis=1)
print(np.mean(v_cmin))

0.037760999999999996


Vậy em chọn câu $[b]$ $0.01$

## Câu 2 (1 điểm)

In [3]:
# Cách 1:
v_c1 = nus[:,0]
print(np.mean(v_c1))

#row = np.arange(nus.shape[0])
v_crand = []
temp = np.random.choice(np.arange(nus.shape[1]),size = nus.shape[0])
for i in temp:
    v_crand.append(temp[i]/1000)
print(np.mean(v_crand))

0.49985
0.48589502000000007


Ta thấy mean(v) của $c_1$  $\thickapprox$ mean(v) của $c_{rand}$ $\thickapprox \mu$ = 0.5

In [4]:
# Cách 2:
def hoeffding (x, muy, n):
    res = True
    for e in np.arange(0.01,0.05,0.01):
        if (abs(np.mean(x)-muy)>e) > 2*np.exp(-2*n*e**2):
            res = False
            break
    return res

print("c1: ", hoeffding(v_c1, 0.5,100000))
print("crand: ", hoeffding(v_crand,0.5,100000))
print("cmin: ", hoeffding(v_cmin,0.5,100000))

c1:  True
crand:  False
cmin:  False


- Với mỗi giá trị của $0<\epsilon<0.5$, ta luôn có $VT < VP$ tương ứng với mỗi coins ($c_1$,$c_{rand}$)
$\Rightarrow$ Thỏa bất đẳng thức Hoeffding

Vậy em chọn câu $[d]$ $c_1$ and $c_{rand}$

## Câu 3 (1 điểm)

Xác suất gặp lỗi khi $h$ xấp xỉ $y$ (có nhiễu). Có hai trường hợp xảy ra khi có nhiễu:
- $h=f(x)$  nhưng $y\neq f(x)$, xác suất là: $(1-\mu)*(1-\lambda)$
- $h\neq f(x)$ nhưng $y=f(x)$, xác suất là: $\mu*\lambda$  
$\Rightarrow$ xác suất lỗi khi $h$ xấp xỉ $y$ là: $P=(1-\mu)*(1-\lambda)+\mu*\lambda$

Vậy, em chọn câu $[e]$ $(1-\lambda)*(1-\mu)+\lambda*\mu$

## Câu 4 (1 điểm)

${(1 - \mu) * (1 - \lambda) + \mu * \lambda}$

= ${1 - \mu - \lambda + 2\mu\lambda}$

= ${1 - \mu - \lambda(1 - 2\mu)}$

- Để h độc lập với ${\lambda}$ thì $\Rightarrow {\lambda = \frac{1}{2}}$
- Tại $\lambda= \frac{1}{2}$, giá trị $\mu$ trong $P$ sẽ bị triệt tiêu nên $h$ sẽ độc lập với $\mu$. Khi đó, nếu $h$ dự đoán đúng $100\%$ thì vẫn sẽ có $50\%$ sai (do nhiễu). Ngược lại, nếu $h$ dự đoán sai $100\%$ thì vẫn sẽ $50\%$ đúng. 

Vậy em chọn câu $[b]$ 0.5

## Câu 5 (1 điểm)

In [5]:
def generate_target_w():
    """
    Generates target_w from two random, uniformly distributed points in [-1, 1] x [-1, 1].
    
    Returns
    -------
    target_w : The vector of parameters of f.
    """
    # Generate two points from a uniform distribution over [-1, 1]x[-1, 1]
    p1 = np.random.uniform(-1, 1, 2)
    p2 = np.random.uniform(-1, 1, 2)
    # Compute the target W from these two points
    target_w = np.array([p1[1]*p2[0] - p1[0]*p2[1], p2[1] - p1[1], p1[0] - p2[0]]).reshape((-1, 1))
    
    return target_w

In [6]:
def generate_data(N, target_w):
    """
    Generates a data set by generating random inputs and then using target_w to generate the 
    corresponding outputs.
    
    Parameters
    ----------
    N : The number of examples.
    target_w : The vector of parameters of f.
    
    Returns
    -------
    X : The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    Y : The vector of outputs.        
    """
    bad_data = True # `bad_data = True` means: data contain points on the target line 
                    # (this rarely happens, but just to be careful)
                    # -> y's of these points = 0 (with np.sign); 
                    #    we don't want this (y's of data must be -1 or 1)
                    # -> re-generate data until `bad_data = False`
    
    while bad_data == True:
        X = np.random.uniform(-1, 1, (N, 2))
        X = np.hstack((np.ones((N, 1)), X)) # Add 'ones' column
        Y = np.sign(np.dot(X, target_w))
        if (0 not in Y): # Good data
            bad_data = False
    
    return X, Y

In [7]:
def run_LR(X, ys):
    """
    Runs PLA.
    
    Parameters
    ----------
    X : The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    ys : The vector of outputs.
    
    Returns
    -------
    w : The vector of parameters of g.
    """
    X_dagger = np.dot(np.linalg.pinv(np.dot(X.T, X)), X.T)
    w = np.dot(X_dagger, ys)
    return w

In [8]:
def mainrun_5_6(n_in):
    """
    Parameters
    ----------
    n_in : The number of training examples.
    """
    num_runs = 1000
    # The average in-sample error of g - the final hypothesis picked by Linear Regression
    avg_ein = 0.0
    # The average out-of-sample error of g
    avg_eout = 0.0
    # Number of out-sample points (estimation purpose)
    n_out = 1000
    
    for r in range(num_runs):
        # Generate target_w
        target_w = generate_target_w()
        # Generate training set
        X_in, Y_in = generate_data(n_in, target_w)
        # Generate out-of-sample dataset
        X_out, Y_out = generate_data(n_out, target_w)
        
        # Run Linear Regression to pick g
        w = run_LR(X_in, Y_in)
        
        # Predict on training set with found w
        predictions_in = np.dot(X_in, w)
        # Predict on out-of-sample set with found w
        predictions_out = np.dot(X_out, w)
        
        # Compute binary error between y_in/ y_out - correct output & predictions
        ein = np.mean(Y_in != np.sign(predictions_in))
        eout = np.mean(Y_out != np.sign(predictions_out))
        
        # Update average error
        avg_ein += (ein * 1.0 / num_runs)
        avg_eout += (eout * 1.0 / num_runs)
    
    # Print results
    print('Average Ein = %f' % (avg_ein))
    print('Average Eout = %f' % (avg_eout))

In [9]:
mainrun_5_6(100)

Average Ein = 0.040850
Average Eout = 0.050308


Vậy em chọn câu $[c]$ 0.01

## Câu 6 (1 điểm)

Vậy em chọn câu $[c]$ 0.01

## Câu 7 (1 điểm)

In [10]:
def run_PLA(X, ys):
    """
    Runs PLA.
    
    Parameters
    ----------
    X :  The matrix of input vectors (each row corresponds to an input vector); the first column of 
         this matrix is all ones.
    ys : The vector of outputs.
    
    Returns
    -------
    iterations : The number of iterations PLA takes to converge.
    """
    # Init w using Linear Regression
    w = run_LR(X, ys)
    iteration = 0
    
    while True:
        iteration += 1
        # Compute sign for each input vector with current w
        h = np.sign(np.dot(X, w))
        
        # Find misclassified if existed
        for i in range(0, X.shape[0]):
            if h[i] != ys[i]:
                change = X[i]*ys[i]
                # Change current w to make X[i] classified
                w = w + change.reshape((-1, 1))
                break
        
        # Check if converged
        if np.array_equal(h, ys) == True:
            break
    
    return iteration

In [11]:
def main_PLA(N):
    """
    Parameters
    ----------
    N : The number of training samples.
    """
    num_runs = 1000
    # The average number of iterations PLA takes to converge
    avg_num_iterations = 0.0
    
    for r in range(num_runs):
        # Generate target_w
        target_w = generate_target_w()
        
        # Generate training set
        X, ys = generate_data(N, target_w)
        
        # Run PLA to completely separates all the in-sample points
        num_iterations = run_PLA(X, ys)
        
        # Update average num_iterations
        avg_num_iterations += (num_iterations * 1.0 / num_runs)
    
    # Print results
    print('Average num iterations = %f' % (avg_num_iterations))

In [12]:
main_PLA(10)

Average num iterations = 5.874000


Vậy em chọn câu $[a]$ 1

## Câu 8 (1 điểm)

In [13]:
def generate_data_with_noise(N, noise):
    """
    Generates a data set by generating random inputs and then using target_w to generate the 
    corresponding outputs.
    
    Parameters
    ----------
    N : int
        The number of examples.
    noise : float
        The percentage of noise in data set.
    
    Returns
    -------
    X : numpy array, shape (N, 3)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    ys : numpy array, shape (N, 1)
        The vector of outputs.        
    """
    bad_data = True # `bad_data = True` means: data contain points on the target line 
                    # (this rarely happens, but just to be careful)
                    # -> y's of these points = 0 (with np.sign); 
                    #    we don't want this (y's of data must be -1 or 1)
                    # -> re-generate data until `bad_data = False`
    
    while bad_data == True:
        X = np.random.uniform(-1, 1, (N, 2))
        X = np.hstack((np.ones((N, 1)), X)) # Add 'ones' column
        ys = np.sign(X[:, 1]**2 + X[:, 2]**2 - 0.6)
        if (0 not in ys): # Good data
            bad_data = False
            
    # Generate simulated noise by ﬂipping the sign of the output randomly
    indices = np.random.choice(np.arange(N), size=int(noise*N), replace=False)
    ys[indices] = -ys[indices]
    
    return X, ys

In [14]:
def mainrun_8(N, noise):
    """
    Parameters
    ----------
    N   : Number of samples.
    noise  : The percentage of noise in data set.
    """
    num_runs = 1000
    # The average test error of g - the final hypothesis picked by Linear Regression
    avg_err = 0.0
    
    for r in range(num_runs):
        # Generate training set with noise & specific target function
        X, Y = generate_data_with_noise(N, noise)
        
        # Run Linear Regression to pick g if not in the input
        w = run_LR(X, Y)
        
        # Prediction on X with w found
        predictions = np.dot(X, w)
        
        # Compute binary error between ys - correct output & predictions
        err = np.mean(Y != np.sign(predictions))
        
        # Update average error
        avg_err += (err * 1.0 / num_runs)
    
    # Print results
    print('avg_err = %f' % (avg_err))

In [15]:
mainrun_8(1000,0.1)

avg_err = 0.506234


Vậy em chọn câu $[d]$ 0.5

## Câu 9 (1 điểm)

In [16]:
def transform_to_nonlinear(X, ys):
    """
    Transform the N = 1000 training data into the nonlinear feature vector.
    
    Parameters
    ----------
    X :  The matrix of input vectors (each row corresponds to an input vector); the first column of 
         this matrix is all ones.
    ys : The vector of outputs.
        
    Returns
    -------
    Z : The matrix of data after transforming (each row corresponds to an input vector);
        the first column of this matrix is all ones.
    ys : The vector of correct outputs.     
    """
    x1x2 = np.array([X[:, 1]*X[:, 2]])
    # x1 square
    x1sqr = np.array([X[:, 1]**2])
    # x2 square
    x2sqr = np.array([X[:, 2]**2])
    Z = np.concatenate((X, x1x2.T, x1sqr.T, x2sqr.T), axis=1)
        
    return Z, ys

In [17]:
num_runs = 100
avg_w = 0.0
N = 1000
noise = 0.1

for r in range(num_runs):
    X, ys = generate_data_with_noise(N, noise)
    Z, ys = transform_to_nonlinear(X, ys)
    avg_w += (run_LR(Z, ys) * 1.0 / num_runs)

print(avg_w)

[-9.98947531e-01 -9.81307007e-04  1.19206600e-03  8.11733430e-03
  1.57652922e+00  1.56082365e+00]


Dựa vào kết quả trên ta thấy kết quả gần đúng nhất với câu a

Vậy em chọn câu $[a]$ $g(x_1,x_2) = sign(-1-0.05x_1+0.08x_2+0.13x_1x_2+1.5x_1^2 + 1.5x_2^2$

## Câu 10 (1 điểm)

In [18]:
def mainrun_10(N,noise):
    num_runs = 1000 
    E_out = 0.0
    
    for r in range(num_runs):        
        # Generate training set
        X, ys = generate_data_with_noise(N, noise)
        Z,temp = transform_to_nonlinear(X, ys)
        
        w = run_LR(Z, ys) 
        
        err = np.mean(ys != np.sign(np.dot(Z, w)))
              
        E_out += (err * 1.0 / num_runs)
    
    print('Eout = %f'% (E_out))

In [19]:
mainrun_10(1000,0.1)

Eout = 0.124164


Vậy em chọn câu $[b]$ 0.1